# **Hercoderen gemeenten 2022**

In 2022 zijn er een aantal gemeenten samengevoegd tot nieuwe gemeenten. Dit werkt door op een aantal plaatsen in de database:

1. In de definitie van de PROTOS, er moeten voor de nieuwe gemeenten Protos worden aangemaakt en voor de oude gemeenten niet meer.
2. In de data voor de Rioolwaterzuiveringsinstallaties (RWZI's). Waarschijnlijk wordt dit gewijzigd aangeleverd door RIVM, maar voor de zekerheid is er code ontwikkeld om dit op te vangen mocht de data in de oude indeling worden aangeleverd.
3. In de Static tabellen met bevolkingsaantallen. Dit start met de tabel Population\_Base en daarvan worden andere Static-tabellen afgeleid. In de Stored Procedure die de data van Stage naar Inter kopieert, wordt de nieuwe gemeentelijke indeling aangebracht m.b.v. een mapping-tabel. Alle tabellen die afhankelijk zijn van VWSInter.Population\_Base zullen nu ook de nieuwe gemeentelijke indeling krijgen. 
4. In één tabel wordt bevolkingsgegevens per gemeente ingelezen in het VWSStage-schema. De stored-procedure die de data kopieert naar VWSStatic worden aangepast om de data in de nieuwe indeling in VWSStatic te schrijven op dezelfde manier als dat voor de Population\_Base tabel gebeurt. 
5. Een tabel met gemeentelijke bevolkingsaantallen wordt direct ingelezen in het VWSStatic schema. Dit wordt aangepast door deze eerst in het VWSStage schema in te lezen en dan middels een Stored Procedure de gegevens in de juiste indeling in het VWSStatic schema te schrijven (zelfde werkwijze als bij 3 en 4). 
6. In één van de views wordt gerefereerd aan de tabel Safety\_Regions\_per\_Municipal. Hierin staat de relatie tussen gemeenten en veiligheidsregios. Toegevoegd wordt dat alleen de meest recente dataset met deze relatie wordt gebruikt, zodat na het inlezen van een dataset met de nieuwe gemeentelijke indeling, deze view ook direct goed staat.

> N.B. voor 6 moet dan wel een nieuw bestand met Safety\_Regions\_per\_Municipal worden klaargezet in ingelezen. 


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/cbs_population.ipynb",
        "src/statics/cbs_inhabitants_per_municipality.ipynb",
        "src/statics/inhabitants_per_municipality.ipynb"
    ]
}
```

# **TABLES**

---


### STATICS

Om de gemeentelijke indeling te kunnen aanpassen naar de gewenste indeling wordt er een mapping tabel gemaakt die de wijzigingen bevat. Met behulp van deze mapping-tabel wordt de data die van VWSTAGE naar VWSSTATIC wordt gekopieerd, direct in de goede gemeentelijke indeling gezet. Dit gebeurt in de Stored Procedure **SP\_CBS\_POPULATION\_BASE\_STATIC**.

N.B. de kolom "Active" in de tabel Recode\_Municipality staat default op 0, wat betekent dat deze grenswijzigingen _NIET_ worden uitgevoerd. Om ze actief te maken moet met een update commando de waarde in de kolom "Active" op 1 worden gezet: 

Update VWSStatic.Recode\_Municipality SET Active = 1

FROM VWSStatic.Recode\_Municipality 

WHERE Date\_Last\_Inserted = (SELECT Max(Date\_Last\_Inserted) )

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE STATIC TABLE(S).....
DROP TABLE IF EXISTS [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING];
GO

-- THIS TABLE CONTAINS THE REQUIRED MAPPING REGARDING THE RECLASSIFICATIONS OF MULTIPLE MUNCIPALITIES.
CREATE TABLE [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1)
   ,DATE_LAST_INSERTED [DATETIME] NOT NULL DEFAULT GETDATE()
   ,[GM_CODE_ORIGINAL] [NVARCHAR](10)
   ,[GM_NAME_ORIGINAL] [NVARCHAR](255)
   ,[GM_CODE_NEW] [NVARCHAR](10)
   ,[GM_NAME_NEW] [NVARCHAR](255)
   ,[VR_CODE] [NVARCHAR](10)
   ,[VR_NAME] [NVARCHAR](255)
   ,[PROVINCE_CODE] [NVARCHAR](10)
   ,[PROVINCE_NAME] [NVARCHAR](255)
   ,[GGD_CODE] [NVARCHAR](10)
   ,[GGD_NAME] [NVARCHAR](255)
   ,[SHARES] [DECIMAL](16,6) NOT NULL DEFAULT (1.0)
   ,[ACTIVE] [INT] NOT NULL DEFAULT(0)
) ON [PRIMARY];
GO

INSERT INTO [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] (
   [GM_CODE_ORIGINAL],
   [GM_NAME_ORIGINAL],
   [GM_CODE_NEW],
   [GM_NAME_NEW],
   [VR_CODE],
   [VR_NAME],
   [PROVINCE_CODE],
   [PROVINCE_NAME],
   [GGD_CODE],
   [GGD_NAME],
   [SHARES],
   [ACTIVE])
VALUES 
   ('GM1684','Cuijk','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
   ,('GM0756','Boxmeer','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0815','Mill en Sint Hubert','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM1702','Sint Anthonis','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0786','Grave','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM1685','Landerd','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0856','Uden','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0398','Heerhugowaard','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,0)
	,('GM0416','Langedijk','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,0)
	,('GM0370','Beemster','GM0439','Purmerend','VR11','Zaanstreek-Waterland','PV27','Noord-Holland','GG7306','GGD Zaanstreek/Waterland',1,0);
GO

# **STORED PROCEDURES**

---


### STAGING &rarr; STATIC MAPPING

> CBS POPULATION BASE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [dbo].[SP_CBS_POPULATION_BASE_STATIC]
AS
BEGIN

-- POPULATION_BASE IS TRANSFERRED FROM STAGE TO STATIC, THE TABLE RECODE_MUNICIPALITY IS USED TO APPLY CHANGES IN THE MUNICIPALITIES.
-- THE RECODE_MUNICIPALITY TABLE IS LEFT-JOINED, SO ONLY MUNICIPALITIES THAT MET WITH A CHANGE WILL BE AFFECTED. 
-- ONLY CHANGES THAT ARE 'ACTIVE' ARE SELECTED.
INSERT INTO [VWSSTATIC].[CBS_POPULATION_BASE]
    (
     [GEMEENTE_CODE]
    ,[GEMEENTE]
    ,[LEEFTIJD]
    ,[GESLACHT]
    ,[DATUM_PEILING]
    ,[POPULATIE]
    ,[VEILIGHEIDSREGIO_CODE]
    ,[VEILIGHEIDSREGIO_NAAM]
    ,[PROVINCIE_CODE]
    ,[PROVINCIE_NAAM]
    ,[GGD_CODE]
    ,[GGD_NAAM]
)
SELECT 
     ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) AS [GEMEENTE_CODE]
    ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) AS [GEMEENTE]
    ,A.[LEEFTIJD]
    ,A.[GESLACHT]
    ,CAST(A.[DATUM_PEILING] AS DATE) AS [DATUM_PEILING]
    ,ROUND(SUM(CAST(A.[POPULATIE] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS [POPULATIE] 
    ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) AS [VEILIGHEIDSREGIO_CODE]
    ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) AS [VEILIGHEIDSREGIO_NAAM]
    ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) AS [PROVINCIE_CODE]
    ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) AS [PROVINCIE_NAAM]
    ,ISNULL(B.[GGD_CODE], A.[GGD_CODE]) AS [GGD_CODE]
    ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) AS [GGD_NAAM]
FROM [VWSSTAGE].[CBS_POPULATION_BASE] A
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
        WHERE [ACTIVE] = 1 
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
    ) B ON A.[GEMEENTE_CODE]=B.[GM_CODE_ORIGINAL]
WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_POPULATION_BASE])
GROUP BY
     ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) 
    ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) 
    ,A.[LEEFTIJD]
    ,A.[GESLACHT]
    ,CAST(A.[DATUM_PEILING] AS DATE) 
    ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) 
    ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) 
    ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) 
    ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) 
    ,ISNULL(B.[GGD_CODE],A.[GGD_CODE]) 
    ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) 
ORDER BY [DATUM_PEILING], ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE])
END;
GO

> INHABITANTS PER REGION

De SP die de data van VWSStage naar VWSStatic kopieert wordt uitgebreid met een join naar de tabel Recode\_Municipality om de gegevens direct om te zetten naar de nieuwe gemeentelijke indeling.

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [dbo].[SP_INHABITANTS_PER_REGION_STATIC]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]  (
        [VRCODE], 
        [GMCODE], 
        [INHABITANTS]
    )
    SELECT
        ISNULL(B.[VR_CODE], [CODE_42]),
        ISNULL(B.[GM_CODE_NEW], [CODE_1]),
        CAST(ROUND(SUM(CAST(A.[INWONERTAL_50] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT)
    FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY] A
        LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE] = 1 
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[CODE_1] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY])
    GROUP BY ISNULL(B.[VR_CODE],[CODE_42]), ISNULL(B.[GM_CODE_NEW],[CODE_1])
END;
GO

> INHABITANTS PER MUNICIPALITY

De tabel INHABITANTS\_PER\_MUNICIPALITY wordt direct in het VWSStatic schema ingelezen. Hiervoor wordt een tabel aangemaakt in het VWSStage schema alsemede een stored procedure om de data van Stage naar Static te kopieren en gedurende de kopieer-actie aan te passen aan de gewenste gemeentelijke indeling. 

Daarnaast moet er nog een Process worden toegevoegd aan de workflow.

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC]
AS
BEGIN

    INSERT INTO [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] (
            [REGIO], 
            [GMCODE], 
            [VRCODE], 
            [VRNAME], 
            [INHABITANTS], 
            [DATE_LAST_INSERTED]
        )
    SELECT  ISNULL(B.[GM_NAME_NEW],A.[REGIO]),
            ISNULL(B.[GM_CODE_NEW],A.[GMCODE]),
            ISNULL(B.[VR_CODE],A.[VRCODE]),
            ISNULL(B.[VR_NAME],A.[VRNAME]),
            CAST(ROUND(SUM(CAST(A.[INHABITANTS] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT),
            MAX(A.[DATE_LAST_INSERTED])
    FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] A
    LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE]=1
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[GMCODE] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY])    
    GROUP BY ISNULL(B.[GM_NAME_NEW],A.[REGIO]),
             ISNULL(B.[GM_CODE_NEW],A.[GMCODE]),
             ISNULL(B.[VR_CODE],A.[VRCODE]),
             ISNULL(B.[VR_NAME],A.[VRNAME])
END;
GO


# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.


### PROTOS

> ACTIVATED PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

DECLARE @proto_id BIGINT,
        @proto_name VARCHAR(50),
		@header_values VARCHAR(256),
		@description VARCHAR(256),
		@is_active INT = 1;

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table
VALUES 
	('GM1982'), -- Land van Cuijk
	('GM1991'), -- Maashorst
	('GM1980'); -- Dijk en Waard

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN
    
	SELECT
		@proto_id = [ID]
	FROM [DATATINO_PROTO_1].[PROTOS]
	WHERE [NAME] = @proto_name;

	SET @header_values = CONCAT(@proto_name, '|', @proto_name, '|', @proto_name ,'|=GETDATE()');
	SET @description = CONCAT('PROTO FILE FOR ',  @proto_name);

	EXEC [DATATINO_PROTO_1].[UPSERT_PROTO]
		@id = @proto_id,
		@proto_name = @proto_name,
		@header_names = 'name|code|proto_name|last_generated',
		@header_values = @header_values,
		@description = @description,
		@active = @is_active;

    FETCH NEXT FROM CUR
    INTO @proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

> DEACTIVATED PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

DECLARE @proto_id BIGINT,
        @proto_name VARCHAR(50),
		@header_values VARCHAR(256),
		@description VARCHAR(256),
		@is_active INT = 0;

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 2) CREATE PROTO(S).....
INSERT INTO @proto_name_table
VALUES 
	('GM1684'), -- Cuijk
	('GM0756'), -- Boxmeer
	('GM0815'), -- Mill en Sint Hubert
	('GM1702'), -- Sint Anthonis
	('GM0786'), -- Grave
	('GM1685'), -- Landerd
	('GM0856'), -- Uden
	('GM0398'), -- Heerhugowaard
	('GM0416'), -- Langedijk
	('GM0370'); -- Beemster (MERGED WITH Purmerend)

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN
    
	SELECT
		@proto_id = [ID]
	FROM [DATATINO_PROTO_1].[PROTOS]
	WHERE [NAME] = @proto_name;

	SET @header_values = CONCAT(@proto_name, '|', @proto_name, '|', @proto_name ,'|=GETDATE()');
	SET @description = CONCAT('PROTO FILE FOR ',  @proto_name);

	EXEC [DATATINO_PROTO_1].[UPSERT_PROTO]
		@id = @proto_id,
		@proto_name = @proto_name,
		@header_names = 'name|code|proto_name|last_generated',
		@header_values = @header_values,
		@description = @description,
		@active = @is_active;

    FETCH NEXT FROM CUR
    INTO @proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

# **OTHER**

---


Hier wordt gebruik gemaakt van de al aanwezige stored procedure `SP_MERGE_MOVE_STATIC_CBS_RWZI` om wijzigingen aan te brengen in de mapping van RWZI's naar gemeenten. Hiervoor wordt er een aantal regels toegevoegd aan het csv-bestand `Aantal_inwoners_per_verzorgingsgebied_mutaties**.csv` in de blob-storage, `datafiles\static` map.

Inhoud van deze csv wordt dan:

```csv 
rwzi_code;rwzi_naam;startdatum;einddatum;inwoners;regio_type;regio_code;regio_naam;aandeel;toelichting

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM1740;Neder-Betuwe;1.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM1740;Neder-Betuwe;0.0%;definitief

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM0214;Buren;98.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM0214;Buren;0.0%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0214;Buren;5.2%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0214;Buren;15.3%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0281;Tiel;75.3%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0281;Tiel;66.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1740;Neder-Betuwe;13.7%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1740;Neder-Betuwe;12.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1960;West Betuwe;5.9%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1960;West Betuwe;5.3%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0297;Zaltbommel;97.8%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0297;Zaltbommel;0.0%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0263;Maasdriel;2.2%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0263;Maasdriel;0.0%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0263;Maasdriel;55.8%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0263;Maasdriel;45.1%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0297;Zaltbommel;44.2%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0297;Zaltbommel;54.9%;definitief

12014;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0370;Beemster;0.0%;definitief

12041;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0439;Purmerend;9.7%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0398;Heerhugowaard;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0416;Langedijk;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM1980;Dijk en Waard;46.7%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0756;Boxmeer;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0815;Mill en Sint Hubert;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1684;Cuijk;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1702;Sint Anthonis;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1982;Land van Cuijk;6.91%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1982;Land van Cuijk;90.07%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM0856;Uden;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1685;Landerd;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0856;Uden;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1685;Landerd;0%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM1991;Maashorst;32.60%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1991;Maashorst;5.64%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1991;Maashorst;9.91%;definitief
```